In [62]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

from pandas.api.types import is_numeric_dtype
from scipy.io.arff import loadarff

raw_data = loadarff('house.arff')
df = pd.DataFrame(raw_data[0])

#print(df['Misc_Feature'].unique())
#Ridondanze: Pool_QC, Misc_Feature(?),
#TODO CONTROLLARE in base anche a feature selection
#df.drop(columns=['Pool_QC'], inplace=True)
#df.drop(columns=['Misc_Val'], inplace=True)

df.drop_duplicates(subset=['Longitude','Latitude'], keep='last',inplace=True)
df.drop(["Latitude","Longitude", "Pool_QC", "Misc_Feature"],axis = 1 , inplace=True)

# drop label columns
X = df.drop(columns=['Sale_Price'])

# isolate y
y = df.loc[:, 'Sale_Price']

#splitter=StratifiedShuffleSplit(n_splits=1,random_state=12) #we can make a number of combinations of split
skf = StratifiedKFold(n_splits=2, random_state=1, shuffle=True)

#But we are interested in only one.

for train,test in skf.split(X,y):     #this will splits the index
    X_train_SS = X.iloc[train]
    y_train_SS = y.iloc[train]
    X_test_SS = X.iloc[test]
    y_test_SS = y.iloc[test]

#X_train_80, X_test, y_train_80, y_test = train_test_split(X, y, random_state=10)

#encoded_X_train_80 = pd.get_dummies(X_train_SS)
#encoded_X_test = pd.get_dummies(X_test_SS)
#display(encoded_X_train_80)
#display(encoded_X_test)

#df = df.select_dtypes([np.object])
#df = df.stack().str.decode('utf-8').unstack()
#print(df.dtypes)

is_numerical = np.array([is_numeric_dtype(X_train_SS[col]) for col in X_train_SS])
#numerical_idx = np.flatnonzero(is_numerical)
categorical_idx = np.flatnonzero(is_numerical==False)
#print(numerical_idx, len(numerical_idx))
print(len(categorical_idx))
print(categorical_idx)

oh = OneHotEncoder(sparse=False, handle_unknown='ignore')
oh.fit(X_train_SS.iloc[:, categorical_idx])

encoded = oh.transform(X_train_SS.iloc[:, categorical_idx])

for i,col in enumerate(oh.get_feature_names()):
    X_train_SS[col] = encoded[:, i]

encoded = oh.transform(X_test_SS.iloc[:, categorical_idx])
for i,col in enumerate(oh.get_feature_names()):
    X_test_SS[col] = encoded[:, i]

print("PRE")
print(len(X_train_SS.columns))
print(len(X_test_SS.columns))

X_train_SS.drop(columns=X_train_SS.columns[categorical_idx], inplace=True)
X_test_SS.drop(columns=X_test_SS.columns[categorical_idx], inplace=True)

print("POST")
print(len(X_train_SS.columns))
print(len(X_test_SS.columns))

display(X_train_SS)

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train_SS,y_train_SS)
best_features = np.argsort(rf.feature_importances_)[::-1]

sum = 0

for bf in best_features:
    sum += rf.feature_importances_[bf]

print(sum)

from sklearn import datasets, linear_model

lasso = linear_model.Lasso()
print(cross_val_score(lasso, X_train_SS, y_train_SS))












"""""
# instantiate the DecisionTreeClassifier model with criterion gini index
clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=100, random_state=0)


# fit the model
clf_gini.fit(X_train_SS, y_train_SS)


y_pred_gini = clf_gini.predict(X_test_SS)


from sklearn.metrics import accuracy_score

print('Model accuracy score with criterion gini index: {0:0.4f}'. format(accuracy_score(y_test_SS, y_pred_gini)))

print(clf_gini)
"""

C:\Users\matti\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\matti\AppData\Local\Temp/ipykernel_9056/1085272729.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_SS[col] = encoded[:, i]
C:\Users\matti\AppData\Local\Temp/ipykernel_9056/1085272729.py:66: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train_SS[col] = encoded

44
[ 0  1  4  5  6  7  8  9 10 11 12 13 14 15 16 17 20 21 22 23 24 26 27 28
 29 30 31 32 34 38 39 40 41 52 54 56 57 58 61 62 63 70 74 75]
PRE
372
372
POST
328
328


C:\Users\matti\AppData\Local\Temp/ipykernel_9056/1085272729.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_SS[col] = encoded[:, i]
C:\Users\matti\AppData\Local\Temp/ipykernel_9056/1085272729.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_SS[col] = encoded[:, i]
C:\Users\matti\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

,Lot_Frontage,Lot_Area,Year_Built,Year_Remod_Add,Mas_Vnr_Area,BsmtFin_SF_1,BsmtFin_SF_2,Bsmt_Unf_SF,Total_Bsmt_SF,First_Flr_SF,...,x42_b'ConLw',x42_b'New',x42_b'Oth',x42_b'WD ',x43_b'Abnorml',x43_b'AdjLand',x43_b'Alloca',x43_b'Family',x43_b'Normal',x43_b'Partial'
0,141.0,31770.0,1960.0,1960.0,112.0,2.0,0.0,441.0,1080.0,1656.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,78.0,9978.0,1998.0,1998.0,20.0,3.0,0.0,324.0,926.0,926.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
6,41.0,4920.0,2001.0,2001.0,0.0,3.0,0.0,722.0,1338.0,1338.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
7,43.0,5005.0,1992.0,1992.0,0.0,1.0,0.0,1017.0,1280.0,1280.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
13,85.0,10176.0,1990.0,1990.0,0.0,3.0,0.0,663.0,1300.0,1341.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2920,21.0,1894.0,1970.0,1970.0,0.0,6.0,0.0,294.0,546.0,546.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2922,63.0,9297.0,1976.0,1976.0,0.0,1.0,0.0,122.0,1728.0,1728.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2925,37.0,7937.0,1984.0,1984.0,0.0,3.0,0.0,184.0,1003.0,1003.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2926,0.0,8885.0,1983.0,1983.0,0.0,2.0,324.0,239.0,864.0,902.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


1.0000000000000002


C:\Users\matti\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 216341484417.61417, tolerance: 669482758.2186916
  model = cd_fast.enet_coordinate_descent(
C:\Users\matti\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 243497964315.3779, tolerance: 717601550.9378058
  model = cd_fast.enet_coordinate_descent(
C:\Users\matti\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 217065470423.546, tolerance: 703985655.6190223
  model = cd_fast.enet_coordinate_descent(
C:\Users\matti\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarnin

[0.90121674 0.90832057 0.71166997 0.62998136 0.92896822]


'""\n# instantiate the DecisionTreeClassifier model with criterion gini index\nclf_gini = DecisionTreeClassifier(criterion=\'gini\', max_depth=100, random_state=0)\n\n\n# fit the model\nclf_gini.fit(X_train_SS, y_train_SS)\n\n\ny_pred_gini = clf_gini.predict(X_test_SS)\n\n\nfrom sklearn.metrics import accuracy_score\n\nprint(\'Model accuracy score with criterion gini index: {0:0.4f}\'. format(accuracy_score(y_test_SS, y_pred_gini)))\n\nprint(clf_gini)\n'

In [41]:





#CON HEADER NONE
X = df.loc[:,1:]
# get class label
y = df.loc[:,0]
print("----------------------------")
#SENZA HEADER A NONE PER USARE NOME COLONNE

df = pd.read_csv('house.csv',sep=';')
X = df.loc[:,'MS_SubClass':]
print('-----------------')
y = df.loc[:,'id']

print(type(df['MS_SubClass'][0]))
#TRY TO COMPUTER LABEL ENCODER
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

le.fit(df['MS_SubClass'])
df['MS_SubClass'] = le.transform(df['MS_SubClass'])
import numpy as np
for column in df:
    if df[column].dtype == 'object':
        le.fit(df[column])
        df[column] = le.transform(df[column])

print(df.head)
X = df.loc[:,'MS_SubClass':]
print('-----------------')
y = df.loc[:,'id']
feature_names = df.columns[1:]
print(feature_names)


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X,y)
print(rf.feature_importances_)

best_features = np.argsort(rf.feature_importances_)[::-1]

rmse = []
from sklearn import datasets, linear_model

lasso = linear_model.Lasso()
print(cross_val_score(lasso, X, y))

----------------------------
count           2931
unique            29
top       North_Ames
freq             443
Name: 12, dtype: object
count        2931
unique         18
top       VinylSd
freq         1015
Name: 24, dtype: object
2          MS_Zoning
5             Street
6              Alley
7          Lot_Shape
8       Land_Contour
9          Utilities
10        Lot_Config
11        Land_Slope
13       Condition_1
14       Condition_2
15         Bldg_Type
16       House_Style
18      Overall_Cond
21        Roof_Style
22         Roof_Matl
25      Mas_Vnr_Type
27        Exter_Qual
28        Exter_Cond
29        Foundation
30         Bsmt_Qual
31         Bsmt_Cond
32     Bsmt_Exposure
33    BsmtFin_Type_1
34      BsmtFin_SF_1
35    BsmtFin_Type_2
39           Heating
40        Heating_QC
41       Central_Air
42        Electrical
47    Bsmt_Full_Bath
48    Bsmt_Half_Bath
49         Full_Bath
50         Half_Bath
51     Bedroom_AbvGr
52     Kitchen_AbvGr
53      Kitchen_Qual
55        F

In [5]:



import pandas as pd
import numpy as np
from tsfresh import extract_features
from tsfresh import feature_selection

df = pd.read_csv('house.csv',sep=';',header=None)
print(df)




#Misc_Val -> valore della feature
#Misc_Feature -> togliamo ??

#Condition 2 -> 2900 Norm su 2931  -> rimuovere

#Roof_Matl -> rimovere troppo uguali
#Piscina con campo a None
#Street -> 13 diversi da Pavel levare

#Utilities -> levare troppi pochi campi diversi

#anno ristrutturazione come unico Anno

"""""
#decidere se tenere una o eliminare tutte le righe con longitudine e latitudine uguale
df = df.drop_duplicates(subset=['Longitude','Latitude'], keep='last',inplace=False)
df.drop(["Latitude","Longitude"],axis = 1 , inplace=True)


columns_to_drop = []
for column in df:
    val = df[column].value_counts().idxmax()
    if df[column].eq(val).mean() > 0.9:
        columns_to_drop.append(column)
df.drop(columns_to_drop, axis=1, inplace=True)
#print(columns_to_drop)
#print(df.info())


"""""
"""""
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(np.unique(df.drop('Lot_Frontage', axis=1)))
X = np.array([le.transform(samp) for samp in df.drop('Lot_Frontage', axis=1).values])
print("-------------------")
print(df['LotFrontage'])
print("-------------------")




# get features by removing id and class
# remove id
X = df.loc[:,]
print("X shape", X.shape)

# get class label
"""""
"""""
y = df.loc[:,]
print("y shape", y.shape)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)


from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

le = LabelEncoder()

le.fit(np.unique(df.drop('Lot_Frontage', axis=1)))
X = np.array([le.transform(samp) for samp in df.drop('Lot_Frontage', axis=1).values])
print("-------------------")
print(df['LotFrontage'])
print("-------------------")




X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

scaler = MinMaxScaler()
scaler.fit(X_train)

"""
from sklearn.model_selection import train_test_split
print(df)
X = df.loc[:,1:]
print("X shape", X.shape)

# get class label
y = df.loc[:,0]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X,y)
rf.feature_importances_


fig, ax = plt.subplots(figsize=(9,4))
ax.bar(range(0,X.shape[1]), rf.feature_importances_)
ax.set_title("Feature Importances")
ax.set_xticks(range(X.shape[1]))
ax.set_xticklabels(df.feature_names)
ax.grid();

"""""
for k in range(1,11):

    kNN = KNeighborsClassifier(n_neighbors=k)
    kNN.fit( scaler.transform(X_train), y_train )
    y_pred = kNN.predict( scaler.transform(X_test) )

    # compute Accuracy
    acc = accuracy_score(y_true=y_test, y_pred=y_pred)
    print ("k: {:2d} | Accuracy {:.3f}".format(k,acc) )

"""

ModuleNotFoundError: No module named 'tsfresh'